In [1]:
from keras import backend as K
from keras.models import load_model
from keras.optimizers import Adam
from scipy.misc import imread
import numpy as np
from matplotlib import pyplot as plt

from models.keras_ssd300 import ssd_300
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization
from data_generator.object_detection_2d_data_generator import DataGenerator
from eval_utils.average_precision_evaluator import Evaluator

%matplotlib inline

C:\Users\c1yeung\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\c1yeung\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.CSRRD7HKRKC3T3YXA7VY7TAZGLSWDKW6.gfortran-win_amd64.dll
C:\Users\c1yeung\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
  stacklevel=1)
Using TensorFlow backend.


In [2]:
# Set a few configuration parameters.
img_height = 300
img_width = 300
n_classes = 200
model_mode = 'inference'

In [3]:
# # 1: Build the Keras model

K.clear_session() # Clear previous models from memory.

model = ssd_300(image_size=(img_height, img_width, 3),
                n_classes=n_classes,
                mode=model_mode,
                l2_regularization=0.0005,
                scales=[0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05], # The scales for MS COCO [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05]
                aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5]],
                two_boxes_for_ar1=True,
                steps=[8, 16, 32, 64, 100, 300],
                offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
                clip_boxes=False,
                variances=[0.1, 0.1, 0.2, 0.2],
                normalize_coords=True,
                subtract_mean=[123, 117, 104],
                swap_channels=[2, 1, 0],
                confidence_thresh=0.01,
                iou_threshold=0.45,
                top_k=200,
                nms_max_output_size=400)

# 2: Load the trained weights into the model.

# TODO: Set the path of the trained weights.
# weights_path = 'VGG_VOC0712_SSD_300x300_iter_120000.h5'
weights_path = 'VGG_coco_SSD_300x300_iter_400000_subsampled_bird_classes.h5'
model.load_weights(weights_path, by_name=True)

# 3: Compile the model so that Keras won't complain the next time you load it.

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [4]:
# model_path = 'withbirds1.h5'

# # We need to create an SSDLoss object in order to pass that to the model loader.
# ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

# K.clear_session() # Clear previous models from memory.

# model = load_model(model_path, custom_objects={'AnchorBoxes': AnchorBoxes,
#                                                'L2Normalization': L2Normalization,
#                                                'DecodeDetections': DecodeDetections,
#                                                'compute_loss': ssd_loss.compute_loss})

In [5]:
with open('./birds/classes.txt','r') as f:
    content = f.readlines()
classes = [x.strip() for x in content] 

In [6]:
val_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)

# TODO: Set the paths to the dataset here.
Pascal_VOC_dataset_images_dir = './VOC2007/VOC2007/JPEGImages/'
Pascal_VOC_dataset_annotations_dir = './VOC2007/VOC2007/Annotations/'
Pascal_VOC_dataset_image_set_filename = './VOC2007/VOC2007/ImageSets/Main/test.txt'

# The XML parser needs to now what object class names to look for and in which order to map them to integers.
# classes = ['background','person','bird','cat','cow','dog','horse','sheep','aeroplane','bicycle',
#           'boat','bus','car','motorbike','train','bottle','chair','diningtable','pottedplant','sofa','tvmonitor']


# dataset.parse_xml(images_dirs=[Pascal_VOC_dataset_images_dir],
#                   image_set_filenames=[Pascal_VOC_dataset_image_set_filename],
#                   annotations_dirs=[Pascal_VOC_dataset_annotations_dir],
#                   classes=classes,
#                   include_classes='all',
#                   exclude_truncated=False,
#                   exclude_difficult=False,
#                   ret=False)

val_dataset.parse_csv(images_dir='./birds/birdimages/',
                    labels_filename='./birds/val.csv',
                    input_format=['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'],
                    include_classes='all')

372


In [7]:
evaluator = Evaluator(model=model,
                      n_classes=n_classes,
                      data_generator=val_dataset,
                      model_mode=model_mode)

results = evaluator(img_height=img_height,
                    img_width=img_width,
                    batch_size=32,
                    data_generator_mode='resize',
                    round_confidences=False,
                    matching_iou_threshold=0.5,
                    border_pixels='include',
                    sorting_algorithm='quicksort',
                    average_precision_mode='sample',
                    num_recall_points=11,
                    ignore_neutral_boxes=True,
                    return_precisions=False,
                    return_recalls=False,
                    return_average_precisions=False,
                    verbose=True)

Number of images in the evaluation dataset: 372

Producing predictions batch-wise: 100%|████████████████████████████████████████████████| 12/12 [06:49<00:00, 31.38s/it]
No predictions for class 1/200
No predictions for class 2/200
No predictions for class 3/200
Matching predictions to ground truth, class 4/200.: 100%|███████████████████████| 1054/1054 [00:00<00:00, 75286.04it/s]
Matching predictions to ground truth, class 5/200.: 100%|█████████████████████████| 407/407 [00:00<00:00, 58145.09it/s]
No predictions for class 6/200
No predictions for class 7/200
Matching predictions to ground truth, class 8/200.: 100%|█████████████████████████| 888/888 [00:00<00:00, 59201.47it/s]
No predictions for class 9/200
No predictions for class 10/200
No predictions for class 11/200
Matching predictions to ground truth, class 12/200.: 100%|████████████████████████| 120/120 [00:00<00:00, 40006.08it/s]
Matching predictions to ground truth, class 13/200.: 100%|██████████████████████| 4094/4094 [00:00<00

No predictions for class 180/200
No predictions for class 181/200
Matching predictions to ground truth, class 182/200.: 100%|███████████████████████| 513/513 [00:00<00:00, 64127.74it/s]
Matching predictions to ground truth, class 183/200.: 100%|█████████████████████| 1971/1971 [00:00<00:00, 75809.02it/s]
No predictions for class 184/200
Matching predictions to ground truth, class 185/200.: 100%|████████████████████████████| 1/1 [00:00<00:00, 1000.31it/s]
Matching predictions to ground truth, class 186/200.: 100%|███████████████████████| 188/188 [00:00<00:00, 62671.21it/s]
No predictions for class 187/200
No predictions for class 188/200
No predictions for class 189/200
No predictions for class 190/200
No predictions for class 191/200
Matching predictions to ground truth, class 192/200.: 100%|███████████████████| 14560/14560 [00:00<00:00, 81800.72it/s]
Matching predictions to ground truth, class 193/200.: 100%|███████████████████| 30002/30002 [00:00<00:00, 79583.45it/s]
No predictions f